In [12]:
from gpuaffman_networks import binary_core
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

sys.path.append("../")
import tasks, ragged_task_evolution

In [2]:
input_states = tasks.make_2_bit_input_state(8)

In [7]:
data_dir_noise = "boolean_network_data/and_evolution_results/2023-03-13-21-10-14"
data_noise = np.load(os.path.join(os.getenv("DATA_DIR"), data_dir_noise, "batch_1.npz"))
use_best = 50
sorted_order = np.argsort(data_noise["errors"])
best_functions = data_noise["functions"][sorted_order][:use_best]
best_connectivity = data_noise["connectivity"][sorted_order][:use_best]
best_used_connectivity = data_noise["used_connectivity"][sorted_order][:use_best]
n_batched_input_state = np.broadcast_to(np.expand_dims(np.expand_dims(input_states, 1), 1), (input_states.shape[0], best_functions.shape[0], 1, input_states.shape[-1]))
errors_n = ragged_task_evolution.evaluate_populations(n_batched_input_state, 1, np.expand_dims(best_functions, 1), np.expand_dims(best_connectivity, 1), np.expand_dims(best_used_connectivity, 1), 10, 0, tasks.evaluate_and_task)
zero_idx = np.where(np.squeeze(errors_n) == 0)
perfs_n = np.squeeze(zero_idx)
best_functions = best_functions[perfs_n]
best_connectivity = best_connectivity[perfs_n]
best_used_connectivity = best_used_connectivity[perfs_n]


In [8]:
data_dir_no_noise = "boolean_network_data/and_evolution_results/no_noise_2023-03-13-20-40-28"
data_no_noise = np.load(os.path.join(os.getenv("DATA_DIR"), data_dir_no_noise, "batch_1.npz"))
nn_best_functions = data_no_noise["functions"]
nn_best_connectivity = data_no_noise["connectivity"]
nn_best_used_connectivity = data_no_noise["used_connectivity"]
nn_batched_input_state = np.broadcast_to(np.expand_dims(np.expand_dims(input_states, 1), 1), (input_states.shape[0], nn_best_functions.shape[0], 1, input_states.shape[-1]))
errors_nn = ragged_task_evolution.evaluate_populations(nn_batched_input_state, 1, np.expand_dims(nn_best_functions, 1), np.expand_dims(nn_best_connectivity, 1), np.expand_dims(nn_best_used_connectivity, 1), 10, 0, tasks.evaluate_and_task)
zero_idx = np.where(np.squeeze(errors_nn) == 0)
perfs_nn = np.squeeze(zero_idx)
nn_best_functions = nn_best_functions[perfs_nn]
nn_best_connectivity = nn_best_connectivity[perfs_nn]
nn_best_used_connectivity = nn_best_used_connectivity[perfs_nn]

In [39]:
def used_function_ind(used_connectivity):
    n_bits = np.shape(used_connectivity)[-1]
    return np.einsum("kj, j, ...j -> ...k", binary_core.truth_table_inputs(n_bits), np.array([1<<x for x in range(n_bits)]), used_connectivity)

def trim_function(function, used_connectivity):
    inds_to_use = list(set(used_function_ind(used_connectivity)))
    return np.array(inds_to_use), function[inds_to_use]

def check_a2(inds, func):
    is_one_inds = np.squeeze(np.argwhere(func), -1)
    is_one_inputs = inds[is_one_inds]
    for i in range(len(is_one_inputs)):
        current = is_one_inputs[i]
        other = np.concatenate([is_one_inputs[:i], is_one_inputs[i+1:]])
        matches = np.bitwise_and(current, other)
        if not np.count_nonzero(matches) == len(matches):
            return False
    return True


def count_a2(all_used_conn, all_functions):
    all_a2 = []
    for used_conn_batch, function_batch in zip(all_used_conn, all_functions):
        this_a2 = []
        for used_conn, function in zip(used_conn_batch, function_batch):
            trim_inds, trim_f = trim_function(function, used_conn)
            this_a2.append(check_a2(trim_inds, trim_f))
        all_a2.append(this_a2)
    all_a2 = np.array(all_a2)
    return  np.sum(all_a2, axis=-1)

n_a2_noise = count_a2(best_used_connectivity, best_functions)/8
n_a2_nn = count_a2(nn_best_used_connectivity, nn_best_functions)/8

noise_p = np.mean(best_functions)
random_functions = np.random.binomial(1, noise_p, (200, 8, 8)).astype(np.bool_)
random_used_conn = np.ones((200, 8, 3)).astype(np.bool_)
n_a2_random = count_a2(random_used_conn, random_functions)/8

In [42]:
%matplotlib tk
def make_cdf(x):
    return sorted(x), np.arange(len(x)) / len(x)

noise_cdf_x, noise_cdf_p = make_cdf(n_a2_noise)
nn_cdf_x, nn_cdf_p = make_cdf(n_a2_nn)
random_cdf_x, random_cdf_p = make_cdf(n_a2_random)

fig, axs = plt.subplots()
axs.plot(noise_cdf_x, noise_cdf_p, label="Noisy AND")
axs.plot(nn_cdf_x, nn_cdf_p, label="Noiseless AND")
axs.plot(random_cdf_x, random_cdf_p, label="Random, same p")
axs.set_xlabel(r"fraction of $A^2$ functions within a network")
axs.set_ylabel("cumulative probability over all populations")
axs.set_ylim([0, 1])
axs.legend()



In [35]:
maj = [0, 0, 0, 1, 0, 1, 1, 1]
inds = np.arange(start=0, stop=len(maj), step=1)
check_a2(inds, maj)


True

In [36]:
nonmaj = [1, 0, 0, 1, 0, 1, 1, 1]
check_a2(inds, nonmaj)


False